In [9]:
import os
import cv2
import numpy as np
import csv

def calculate_asymmetry_score(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Threshold the grayscale image
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Find the contour with the largest area
    contour = max(contours, key=cv2.contourArea)
    # Calculate the moments of the contour
    M = cv2.moments(contour)
    # Calculate the centroid
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])

    # Calculate the distance between each contour point and the centroid
    distances = [np.sqrt((cx - point[0][0])**2 + (cy - point[0][1])**2) for point in contour]
    # Calculate asymmetry score
    asymmetry_score = np.std(distances) / np.mean(distances)

    return asymmetry_score

def calculate_color_variation(image):
    # Convert image to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    # Split the LAB image into channels
    l, a, b = cv2.split(lab)
    # Calculate standard deviation of each channel
    std_l = np.std(l)
    std_a = np.std(a)
    std_b = np.std(b)
    # Calculate color variation
    color_variation = np.sqrt(std_l**2 + std_a**2 + std_b**2)

    return color_variation

def calculate_compactness_index(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Find contours
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Find the contour with the largest area
    contour = max(contours, key=cv2.contourArea)
    # Calculate area and perimeter of the contour
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    # Calculate compactness index
    compactness_index = (perimeter**2) / (4 * np.pi * area)

    return compactness_index

def extract_features_from_folder(folder_path):
    features = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            if image is not None:
                asymmetry_score = calculate_asymmetry_score(image)
                color_variation = calculate_color_variation(image)
                compactness_index = calculate_compactness_index(image)
                features.append((filename, asymmetry_score, color_variation, compactness_index))
    return features

def write_to_csv(features, csv_filename):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Filename', 'Asymmetry Score', 'Color Variation', 'Compactness Index'])
        for feature in features:
            writer.writerow(feature)

# Paths to folders containing images for benign and malignant classes
benign_folder = "/content/Bneign_seg/"
malignant_folder = "/content/Malignant_seg/"

# Extract features from both folders
benign_features = extract_features_from_folder(benign_folder)
malignant_features = extract_features_from_folder(malignant_folder)

# Write features to CSV files
write_to_csv(benign_features, 'Benign_features.csv')
write_to_csv(malignant_features, 'Malignant_features.csv')

print("Features saved to CSV files: benign_features.csv, malignant_features.csv")

Features saved to CSV files: benign_features.csv, malignant_features.csv
